# 第6章 对象引用、可变性和垃圾回收

## 6.1 浅拷贝

浅拷贝的方式：

- 复制列表使用内置的类型构造函数。
- 使用`[:]`进行浅拷贝。
- 使用`copy.copy`函数进行浅拷贝。

In [1]:
l1 = [3, [66, 55, 44], (7, 8, 9)]
l2 = list(l1)
l1.append(100)
l1[1].remove(55)
print('l1:', l1)
print('l2:', l2)
l2[1] += [33, 22]
l2[2] += (10, 11)
print('l1:', l1)
print('l2:', l2)

l1: [3, [66, 44], (7, 8, 9), 100]
l2: [3, [66, 44], (7, 8, 9)]
l1: [3, [66, 44, 33, 22], (7, 8, 9), 100]
l2: [3, [66, 44, 33, 22], (7, 8, 9, 10, 11)]


![浅拷贝可视化](./images/06-01-python-vis.png)

深拷贝方式：使用`copy.deepcopy`函数

In [2]:
# 校车乘客在途中有上有下
class Bus:
    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = list(passengers)

    def pick(self, name):
        self.passengers.append(name)

    def drop(self, name):
        self.passengers.remove(name)

In [6]:
import copy
bus1 = Bus(['Alice', 'Bill', 'Claire', 'David'])
# 使用浅拷贝
bus2 = copy.copy(bus1)
# 使用深拷贝
bus3 = copy.deepcopy(bus1)
bus1.drop('Bill')
bus2.passengers

['Alice', 'Claire', 'David']

In [7]:
bus3.passengers

['Alice', 'Bill', 'Claire', 'David']

## 6.2 防御可变参数

In [8]:
class TwilightBus:
    """让乘客销声匿迹的校车"""

    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []  
        else:
            # WARN：不能直接引用变量，应使用浅拷贝
            self.passengers = passengers

    def pick(self, name):
        self.passengers.append(name)

    def drop(self, name):
        self.passengers.remove(name)

In [10]:
basketball_team = ['Sue', 'Tina', 'Maya', 'Diana', 'Pat']
bus = TwilightBus(basketball_team)
bus.drop('Tina')
bus.drop('Pat')
# 下车的学生从篮球队中消失了！
basketball_team

['Sue', 'Maya', 'Diana']

In [11]:
# 修改后的代码
class TwilightBus:
    """让乘客销声匿迹的校车"""

    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []  
        else:
            # 使用构造器的浅拷贝方式
            self.passengers = list(passengers)

    def pick(self, name):
        self.passengers.append(name)

    def drop(self, name):
        self.passengers.remove(name)

## 6.3 本章小结

- 简单的赋值不创建副本，而是建立引用。
- 对`+=`或`*=`增量赋值，如果左边的变量绑定的是不可变对象，则创建新对象，如果是可变对象，就地修改。
- 为现有的变量赋予新值，不修改之前绑定的变量。如果变量是之前那个对象的最后一个引用，重新绑定了其他对象，则对象被当作垃圾回收。
- 函数的形参以别名的形式传递，这意味着函数可能会修改通过实参传入的可变对象。
- 使用可变类型作为函数参数的默认值，是危险的。如果修改了参数，默认值也会发生变化。

## 6.4 杂谈

- 平等对待所有对象：Python采用的方式是`==`运算符比较对象的值，而`is`比较引用。
- 可变性：在Python中，用户定义的类，实例默认可变。
- 对象析构和垃圾回收：CPython的垃圾回收主要依靠引用计数，在CPython2.0实现了分代垃圾回收机制，把引用循环中不可达的对象销毁。
- 参数传递（共享传参）：
    - 在早期的语言中，参数传递模式有按值传递（函数得到实参的副本）和按引用传递（函数得到实参的指针）。
    - 在Python中，函数得到实参的副本，但是实参始终是引用的。